<a href="https://colab.research.google.com/github/apoorva2810/CS6910/blob/main/Assignment-2/partA/partA_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Before executing the code, allocate GPU in Colab from: Edit -> Notebook Settings -> Hardware accelerator -> GPU

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout
import random
import os
from google.colab import drive
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm
from PIL import Image
import pickle
from keras.models import Model
from keras.layers import Input
from tensorflow.keras import layers, models, Model, optimizers
import tensorflow.keras as K

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing  
from keras.layers import LeakyReLU
from keras.models import Model



In [ ]:
#To access content from google drive
drive.mount('/content/gdrive')

In [ ]:
#Todo- first rename the file in drive
#Use this link: https://drive.google.com/drive/folders/1yu5DuYlM_cvV5qwIKKnl3sHRG_G41aLB?usp=sharing
#To make copy of pickled data and save it folder name 'Raw_data350'

with open('/content/gdrive/My Drive/Raw_data350/xtrain','rb') as xT:
  xTrain = pickle.load(xT)

with open('/content/gdrive/My Drive/Raw_data350/ytrain','rb') as yT:
  yTrain = pickle.load(yT)

with open('/content/gdrive/My Drive/Raw_data350/xtest','rb') as xV:
  xTest = pickle.load(xV)

with open('/content/gdrive/My Drive/Raw_data350/ytest','rb') as yV:
  yTest = pickle.load(yV)

In [ ]:
xplot=[]
yplot=[]
x=10
for i in range(30):
  xplot.append(xTest[x])
  yplot.append(yTest[x])
  x=x+25


In [ ]:
len(yTest)

In [ ]:
xTrain, xVal, yTrain, yVal= train_test_split(xTrain, yTrain, test_size=.1)

In [ ]:
len(xTrain)

In [ ]:
#Removing all the images which have only 2 dimensions and their respective y label from Train data

for i in range(len(xTrain)):

  xTrain[i]=np.array(xTrain[i])
  
  if(xTrain[i].ndim==2):
    del xTrain[i]
    del yTrain[i]

xTrain=np.array(xTrain)


In [ ]:
#Removing all the images which have only 2 dimensions and their respective y label from Validation data

for i in range(len(xVal)):

  xVal[i]=np.array(xVal[i])
  
  if(xVal[i].ndim==2):
    del xVal[i]
    del yVal[i]

xVal=np.array(xVal)

In [ ]:
#Removing all the images which have only 2 dimensions and their respective y label from Train data

for i in range(len(xTest)):

  xTest[i]=np.array(xTest[i])
  
  if(xTest[i].ndim==2):
    del xTest[i]
    del yTest[i]

xTest=np.array(xTest)
type(xTrain)

In [ ]:
# Normalising xTrain
for i in range(len(xTrain)):
  # xTrain[i]= xTrain[i].astype('float32')
  xTrain[i]= xTrain[i]/255.0

In [ ]:
# Normalising xVal
for i in range(len(xVal)):

  xVal[i]=xVal[i].astype('float32')
  xVal[i]=xVal[i]/255.0

In [ ]:
# Normalising xTest
for i in range(len(xTest)):

  xTest[i]=xTest[i].astype('float32')
  xTest[i]=xTest[i]/255.0

In [ ]:
# Assigning the string labels of yTest and yTrain to integers
pple = preprocessing.LabelEncoder()
yTest=pple.fit_transform(yTest)
yVal=pple.fit_transform(yVal)
yTrain=pple.fit_transform(yTrain)

In [ ]:
#Change the integer y to one-hot vector
def to_categorical(y, n_class):
    return np.eye(n_class, dtype='uint8')[y]

In [ ]:
# getting the categorical value of yTrain and yTest using to_categorical() function
yTrain_cat=[]
for i in range(len(yTrain)):
  cat_d=to_categorical(int(yTrain[i]),10)
  yTrain_cat.append(cat_d)

yVal_cat=[]
for i in range(len(yVal)):
  cat_d=to_categorical(int(yVal[i]),10)
  yVal_cat.append(cat_d)

yTest_cat=[]
for i in range(len(yTest)):
  cat_d=to_categorical(int(yTest[i]),10)
  yTest_cat.append(cat_d)

In [ ]:
#changing the yTrain_cat and yTest_cat to numpy array 

yTrain_cat=np.array(yTrain_cat)
yVal_cat=np.array(yVal_cat)
yTest_cat=np.array(yTest_cat)

In [ ]:
print(yTrain_cat.shape)
print(yVal_cat.shape)
print(yTest_cat.shape)